DensNet Model

In [ ]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
data = pd.read_csv('E:\\Machine-Learning\\Project\\Cleaned_Lands.csv')
print(data.head())

    Lands     Land_location  Land_area Special_location Soil_type  \
0  Land_1  New Capital City      383.0              Yes    Gravel   
1  Land_2  New Capital City      384.0              Yes    Gravel   
2  Land_3  New Capital City      385.0              Yes    Gravel   
3  Land_4  New Capital City      386.0              Yes    Gravel   
4  Land_5  New Capital City      387.0              Yes    Gravel   

  Utilities_Availability Pollution  Land_price  
0                    Yes       Low    30000000  
1                    Yes       Low    30000001  
2                    Yes       Low    30000002  
3                    Yes       Low    30000003  
4                    Yes       Low    30000004  


In [ ]:
# Constants
IMAGE_SIZE = (224, 224)  # Resize images to 224x224
BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 0.001

In [2]:
# Load and Preprocess Data
def load_data(image_dir):
    """Load images from a directory and preprocess them."""
    images = []
    labels = []
    for label_dir in os.listdir(image_dir):
        label_path = os.path.join(image_dir, label_dir)
        if os.path.isdir(label_path):
            for image_file in os.listdir(label_path):
                image_path = os.path.join(label_path, image_file)
                try:
                    # Load and preprocess image
                    img = load_img(image_path, target_size=IMAGE_SIZE)
                    img_array = img_to_array(img)
                    images.append(img_array)
                    labels.append(label_dir)
                except Exception as e:
                    print(f"Error loading image {image_path}: {e}")
    return np.array(images), np.array(labels)


In [1]:
# Build the Model

def build_model():
    """Create a CNN model for image classification."""
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # For binary classification; change to softmax for multiclass
    ])
    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
                  loss='binary_crossentropy',  # Change to 'sparse_categorical_crossentropy' for multiclass
                  metrics=['accuracy'])
    return model

X_train: [[7, 8], [1, 2], [5, 6]]
X_test: [[3, 4]]


In [ ]:
# Train the model

def train_model(model, X_train, y_train, X_val, y_val):
    """Train the model with the given data."""
    # Data augmentation
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True
    )
    datagen.fit(X_train)

    # Early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Train the model
    history = model.fit(
        datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
        validation_data=(X_val, y_val),
        epochs=EPOCHS,
        callbacks=[early_stopping]
    )
    return history

In [4]:
# Main

if __name__ == "__main__":
    # Path to your dataset
    image_dir = r"E:\Machine-Learning\Project\Images"

    # Step 1: Load and preprocess data
    print("Loading data...")
    X, y = load_data(image_dir)
    X = X / 255.0  # Normalize pixel values
    y = np.array([1 if label == "class1" else 0 for label in y])  # Binary labels for simplicity

    # Split the data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Step 2: Build the model
    print("Building model...")
    model = build_model()
    model.summary()

    # Step 3: Train the model
    print("Training model...")
    history = train_model(model, X_train, y_train, X_val, y_val)

    # Step 4: Save the trained model
    print("Saving model...")
    model.save(r"E:\Machine-Learning\Project\image_classification_model.h5")
    print("Model saved as 'image_classification_model.h5'.")